In [1]:
import json
import sqlite3

In [6]:
con = sqlite3.connect("local.db")
cur = con.cursor()

In [7]:
cur.execute("CREATE TABLE Book (\
    abbrev          CHAR (2)  PRIMARY KEY\
                              UNIQUE\
                              NOT NULL,\
    full_name       CHAR (50) NOT NULL,\
    number_in_order INTEGER   UNIQUE\
                              NOT NULL\
);\
")
cur.execute("CREATE TABLE Chapter (\
    id      INTEGER  PRIMARY KEY AUTOINCREMENT\
                     UNIQUE\
                     NOT NULL,\
    book_of CHAR (2) REFERENCES Book (abbrev) \
                     NOT NULL,\
    number  INTEGER  NOT NULL\
);\
")
cur.execute("CREATE TABLE Verse (\
    id         INTEGER PRIMARY KEY AUTOINCREMENT\
                       NOT NULL\
                       UNIQUE,\
    chapter_of INTEGER REFERENCES Chapter (id) \
                       NOT NULL,\
    number     INTEGER NOT NULL,\
    text       TEXT    NOT NULL\
);\
")
con.commit()

In [8]:
with open("ru.json", encoding="utf-8") as f:
    Bible = json.load(f)
with open("transitions.json", encoding="utf-8") as f:
    abbrevs = json.load(f)

In [9]:
for book_num, book in enumerate(Bible):
    cur.execute(f"INSERT INTO Book (abbrev, full_name, number_in_order) VALUES ('{book}', '{abbrevs[book]}', {book_num + 1})")
    for chapter, ch_text in enumerate(Bible[book]):
        cur.execute(f"INSERT INTO Chapter (book_of, number) VALUES ('{book}', {chapter + 1})")
        chapter_of = cur.lastrowid
        for verse, ve_text in enumerate(ch_text):
            cur.execute(f"INSERT INTO Verse (chapter_of, number, text) VALUES ({chapter_of}, {verse + 1}, '{ve_text}')")
con.commit()

In [29]:
cur.execute("SELECT v.text FROM Verse v WHERE v.chapter_of = (SELECT c.id FROM Chapter c WHERE c.book_of = 'бт' AND c.number = 1) AND v.number = 1;").fetchall()[0][0]

'В начале сотворил Бог небо и землю.'

In [ ]:
""" Поиск стиха SELECT v.text FROM Verse v WHERE v.chapter_of = (
SELECT c.id FROM Chapter c WHERE c.book_of = 'бт' AND c.number = 1) AND v.number = 1
LIMIT 1600; """

""" Поиск текста -- SELECT v.text FROM Verse v WHERE v.text LIKE '%не бойся%'; """
